In [26]:
import pandas as pd
df_rtm = pd.read_csv('../data/Simulated_RTM_Logs.csv')
df_ipdr = pd.read_csv('../data/Simulated_CDR_IPDR_Logs.csv')

df_rtm['date'] = pd.to_datetime(df_rtm['timestamp']).dt.date
df_ipdr['date'] = pd.to_datetime(df_ipdr['timestamp']).dt.date

df_rtm['hour'] = pd.to_datetime(df_rtm['timestamp']).dt.hour
df_ipdr['hour'] = pd.to_datetime(df_ipdr['timestamp']).dt.hour

rtm_summary = df_rtm.groupby(['subscriber_id', 'date', 'hour'])['duration_sec'].sum().reset_index(name="app_duration")

ipdr_summary = df_ipdr.groupby(['subscriber_id', 'date', 'hour'])['duration_sec'].sum().reset_index(name="cdr_ipdr_duration")

In [28]:
merged_by_date_and_hour = rtm_summary.merge(ipdr_summary, on=['subscriber_id', 'date', 'hour'], how='outer')
merged_by_date_and_hour

,subscriber_id,date,hour,app_duration,cdr_ipdr_duration
0,SUB_001,2025-04-01,0,113.0,NaN
1,SUB_001,2025-04-01,1,2694.0,NaN
2,SUB_001,2025-04-01,3,595.0,NaN
3,SUB_001,2025-04-01,5,641.0,984.0
4,SUB_001,2025-04-01,6,1644.0,NaN
...,...,...,...,...,...
469,SUB_010,2025-04-03,17,500.0,NaN
470,SUB_010,2025-04-03,19,876.0,NaN
471,SUB_010,2025-04-03,20,2451.0,NaN
472,SUB_010,2025-04-03,21,82.0,NaN
